# Simulating and fitting data using Ball & Stick model

As in the previous tutorial we first create an acquisition scheme that we will use to simulate and fit data.

In [1]:
# load the necessary modules
from microstruktur.signal_models import cylinder_models, gaussian_models
from microstruktur.core import modeling_framework
from microstruktur.acquisition_scheme.acquisition_scheme import acquisition_scheme_from_bvalues
from os.path import join
import numpy as np

acquisition_path = modeling_framework.GRADIENT_TABLES_PATH
bvalues_SI = np.loadtxt(join(acquisition_path, 'bvals_hcp_wu_minn.txt')) * 1e6  # in s/m^2
gradient_directions = np.loadtxt(join(acquisition_path, 'bvecs_hcp_wu_minn.txt'))  # on unit sphere
delta = 0.0106 # in seconds
Delta = 0.0431 # in seconds
acq_scheme = acquisition_scheme_from_bvalues(
    bvalues_SI, gradient_directions, delta, Delta)

Now to make our first multi-compartment Microstructure model! Don't worry, we start with a simple Ball & Stick model, and the procedure will be very simple and similar as before. First, we instantiate the models we want:

In [2]:
stick = cylinder_models.C1Stick()
ball = gaussian_models.G3Ball()

Next, we combine them into one model using PartialVolumeCombinedMicrostructureModel.  
Our new model will now be $E = vf * E_{Ball} + (1 - vf) * E_{Stick}$ where $vf$ is the volume fraction between zero and one.

In [3]:
ball_and_stick = (
    modeling_framework.MultiCompartmentMicrostructureModel(
        models=[ball, stick]))

That it! We can again figure out the required parameter names using parameter_cardinality. As we are combining models, the parameter names now have the corresponding model prepended to it.

In [4]:
ball_and_stick.parameter_cardinality

OrderedDict([('G3Ball_1_lambda_iso', 1),
             ('C1Stick_1_mu', 2),
             ('C1Stick_1_lambda_par', 1),
             ('partial_volume_0', 1)])

It can be seen that now 'partial_volume_0' also appears. Note that the ordering in which you give models matters for the significance of the partial_volume_0! The procedure to simulate data is now the same as before:

In [5]:
mu = (np.pi / 2., np.pi / 2.)
lambda_par = 1e-9
lambda_iso = .6e-9
partial_volume = 0.5

parameter_vector = ball_and_stick.parameters_to_parameter_vector(
    C1Stick_1_lambda_par=lambda_par,
    G3Ball_1_lambda_iso=lambda_iso,
    C1Stick_1_mu=mu,
    partial_volume_0=partial_volume
)

E = ball_and_stick.simulate_signal(
    acq_scheme, parameter_vector)

Fitting the data is also the same as before:

In [6]:
initial_mu = (np.pi / 4., np.pi / 4.)
initial_lambda_par = 1.4e-9
intial_lambda_iso = .2e-9
intial_partial_volume = .3

x0 = ball_and_stick.parameters_to_parameter_vector(
    C1Stick_1_lambda_par=initial_lambda_par,
    G3Ball_1_lambda_iso=intial_lambda_iso,
    C1Stick_1_mu=initial_mu,
    partial_volume_0=intial_partial_volume
)
res = ball_and_stick.fit(E, acq_scheme, x0)

In [7]:
# the optimized results
ball_and_stick.parameter_vector_to_parameters(res)

{'C1Stick_1_lambda_par': array([  9.99934872e-10]),
 'C1Stick_1_mu': array([ 1.57084068,  1.57082972]),
 'G3Ball_1_lambda_iso': array([  6.00003325e-10]),
 'partial_volume_0': array([ 0.49997512])}

In [8]:
# and the ground truth correspond
ball_and_stick.parameter_vector_to_parameters(parameter_vector)

{'C1Stick_1_lambda_par': array([  1.00000000e-09]),
 'C1Stick_1_mu': array([ 1.57079633,  1.57079633]),
 'G3Ball_1_lambda_iso': array([  6.00000000e-10]),
 'partial_volume_0': array([ 0.5])}